<img src="計算流程.JPG" style="width:935px;height:500px;float:middle">
以上為此程式的計算流程<br>

### 請解壓縮data.rar，取得本程式之數據

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import cv2
import os

C:\Users\hong guo peng\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# 超參數
EPOCHS = 50
LR = 1e-3
BS = 32
IMAGE_DIMS = [96 , 96 , 3]

## 讀取檔案之前，請直接解壓縮dataset.rar

In [3]:
# 獲取數據路徑
image_types = ['.jpg' , '.jpeg' , '.png' , '.bmp' , '.tif' , '.tiff'] # 只要副檔名符合image_types中的任何一個都當作訓練資料    
imagePaths = [] 
for files in os.listdir('./dataset'):
    for image in os.listdir('./dataset/{}'.format(files)):
        # os.path.splitext(image) => 獲取image的副檔名
        if os.path.splitext(image)[-1].lower() in image_types:
            imagePaths.append('./dataset/{}/{}'.format(files , image))
random.seed(42)
random.shuffle(imagePaths)


# 獲取數據標簽
data , labels = [] , []
for imagePath in imagePaths:
    # 讀取image，並將image做resize
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1] , IMAGE_DIMS[0]))
    data.append(image)
    
    # 服裝顏色 => label[0]    服裝種類 => label[1]  
    label = imagePath.split('/')[-2].split('_')
    labels.append(label)     

    
# 將服裝顏色與服裝種類的label分開處理
label_color , label_clothing = [] , []
for (color , clothing) in labels:
    label_color.append(color)
    label_clothing.append(clothing)
    
     
# 將讀取到的image做預處理 
data = np.array(data , dtype = 'float') / 255.0
data = data.astype('float32')
label_clothing = np.array(label_clothing)
label_color = np.array(label_color)


# 制作標簽
# 本程式在此將用2組neural network分別預測服裝顏色與服裝種類
# 其實就是分別做服裝顏色與服裝種類的multi classification
label_clothing_onehot = pd.get_dummies(label_clothing)
label_color_onehot = pd.get_dummies(label_color)
label_clothing_onehot = np.array(label_clothing_onehot).astype('float32')
label_color_onehot = np.array(label_color_onehot).astype('float32')

# 數據集切分
(trainX , testX , trainY , testY) = train_test_split(data,
                                                     np.concatenate([label_clothing_onehot , label_color_onehot] , axis = 1), 
                                                     test_size = 0.2, 
                                                     random_state = 42)

In [4]:
def weight_variable(shape):
    initializer = tf.truncated_normal(shape , stddev = 0.0001)
    return tf.Variable(initializer)
    
def bias_variable(shape):
#    initial = tf.zeros(shape)
    initial = tf.constant(0.0001 , shape = shape) 
    return tf.Variable(initial)

def conv2d(x , W):
    # stride [1, x_movement , y_movement, 1]
    # Must have strides[0] = strides[3] = 1	
    return tf.nn.conv2d(x , W , strides = [1 , 1 , 1 , 1] , padding = 'SAME') 

def max_pool(x , k , s):
    # 不需要跟tf.nn.conv2d一樣要輸入W
    # ksize = [1 , *2* , *2* , 1] 輸入 2 , 2 代表每2x2個pixel做一次選取pixel最大的動作
    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x , ksize = [1 , k , k , 1] , strides = [1 , s , s , 1] , padding = 'SAME')

def batch_norm_layer(inputs , on_train , convolution):
    # the dimension you wanna normalize, here [0] for batch
    # for image, you wanna do [0 , 1 , 2] for [batch , height , width] but not channel
    if convolution:
        fc_mean , fc_var = tf.nn.moments(inputs , axes = [0 , 1 , 2] , name = 'mean_var')
    else:
        fc_mean , fc_var = tf.nn.moments(inputs , axes = [0] , name = 'mean_var')
    
    ema = tf.train.ExponentialMovingAverage(decay = 0.75)
    ema_apply_op = ema.apply([fc_mean , fc_var])
    mean = tf.cond(on_train , lambda : fc_mean , lambda : ema.average(fc_mean))
    var = tf.cond(on_train , lambda : fc_var , lambda : ema.average(fc_var))
    
    scale = tf.Variable(tf.ones([1 , inputs.shape[-1].value]) , name = 'scale') 
    shift = tf.Variable(tf.zeros([1 , inputs.shape[-1].value]) , name = 'shift')
    temp = (inputs - mean) / tf.sqrt(var + 0.001)
    outputs = tf.multiply(temp , scale) + shift
  
    return outputs , ema_apply_op 

In [5]:
# define placeholder for inputs to network
xs = tf.placeholder(tf.float32 , [None , IMAGE_DIMS[0] , IMAGE_DIMS[0] , IMAGE_DIMS[2]])
ys = tf.placeholder(tf.float32 , [None , 6])
on_train = tf.placeholder(tf.bool) # train/test selector for dropout & batch normalization
lr = tf.placeholder(tf.float32)

In [6]:
# 針對服裝顏色的multi classification
def classfication_for_color(xs , ys , on_train , lr):
    # conv1 layer 
    conv_w_1 = weight_variable([3 , 3 , 3 , 16]) 
    conv_b_1 = bias_variable([16])
    conv_output_1 = tf.nn.relu(conv2d(xs , conv_w_1) + conv_b_1) 
    conv_bn_1 , conv_ema_1 = batch_norm_layer(conv_output_1 , on_train , True)
    conv_pooling_1 = max_pool(conv_bn_1 , k = 3 , s = 3)
        
    # conv2 layer #
    conv_w_2 = weight_variable([3 , 3 , 16 , 32]) 
    conv_b_2 = bias_variable([32])
    conv_output_2 = tf.nn.relu(conv2d(conv_pooling_1 , conv_w_2) + conv_b_2)
    conv_bn_2 , conv_ema_2 = batch_norm_layer(conv_output_2 , on_train , True)
    conv_pooling_2 = max_pool(conv_bn_2 , k = 2 , s = 2)
    
    # conv3 layer 
    conv_w_3 = weight_variable([3 , 3 , 32 , 32]) 
    conv_b_3 = bias_variable([32])
    conv_output_3 = tf.nn.relu(conv2d(conv_pooling_2 , conv_w_3) + conv_b_3) 
    conv_bn_3 , conv_ema_3 = batch_norm_layer(conv_output_3 , on_train , True)
    conv_pooling_3 = max_pool(conv_bn_3 , k = 2 , s = 2)
    conv_pooling_3_flatten = tf.layers.flatten(conv_pooling_3)
    
    
    # fully connected layer 1
    fc_w_1 = weight_variable([conv_pooling_3_flatten.shape[1].value , 128])
    fc_b_1 = bias_variable([128])
    fc_output_1 = tf.nn.relu(tf.matmul(conv_pooling_3_flatten , fc_w_1) + fc_b_1)
    fc_bn_1 , fc_ema_1 = batch_norm_layer(fc_output_1 , on_train , False)
    fc_dropout_1 = tf.cond(on_train , 
                           lambda : tf.nn.dropout(fc_bn_1 , keep_prob = 0.5) , 
                           lambda : tf.nn.dropout(fc_bn_1 , keep_prob = 1))
        
    # fully connected layer 2
    fc_w_2 = weight_variable([128 , 3])
    fc_b_2 = bias_variable([3])
    prediction = tf.nn.softmax(tf.matmul(fc_dropout_1 , fc_w_2) + fc_b_2)
    
    ys_color = ys[: , 3:]
    cross_entropy = ys_color * tf.log(prediction + 1e-9)  
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(cross_entropy , axis = 1))
    correct = tf.equal(tf.cast(tf.greater_equal(prediction , 0.5) , tf.int32) , tf.cast(ys_color , tf.int32))
    accuracy = tf.reduce_mean(tf.reduce_min(tf.cast(correct , tf.float32) , 1))
    
    ema_list = [conv_ema_1 , conv_ema_2 , conv_ema_3 , fc_ema_1]
    update_ema = tf.group(ema_list)
    optimizer = tf.train.AdamOptimizer(lr)
    grads_and_vars = optimizer.compute_gradients(cross_entropy)
    train_op = optimizer.apply_gradients(grads_and_vars)

    return cross_entropy , accuracy , update_ema , train_op

In [7]:
# 針對服裝種類的multi classification
# 直覺來說，由於服裝顏色的預測較為困難，所以層數會比針對服裝種類的multi classification的還要多
def classfication_for_clothing(xs , ys , on_train , lr):
    # conv1 layer 
    conv_w_1 = weight_variable([3 , 3 , 3 , 32]) 
    conv_b_1 = bias_variable([32])
    conv_output_1 = tf.nn.relu(conv2d(xs , conv_w_1) + conv_b_1) 
    conv_bn_1 , conv_ema_1 = batch_norm_layer(conv_output_1 , on_train , True)
    conv_pooling_1 = max_pool(conv_bn_1 , k = 3 , s = 3)
       
    # conv2 layer #
    conv_w_2 = weight_variable([3 , 3 , 32 , 64]) 
    conv_b_2 = bias_variable([64])
    conv_output_2 = tf.nn.relu(conv2d(conv_pooling_1 , conv_w_2) + conv_b_2)
    conv_bn_2 , conv_ema_2 = batch_norm_layer(conv_output_2 , on_train , True)
    
    # conv3 layer 
    conv_w_3 = weight_variable([3 , 3 , 64 , 64]) 
    conv_b_3 = bias_variable([64])
    conv_output_3 = tf.nn.relu(conv2d(conv_bn_2 , conv_w_3) + conv_b_3) 
    conv_bn_3 , conv_ema_3 = batch_norm_layer(conv_output_3 , on_train , True)
    conv_pooling_3 = max_pool(conv_bn_3 , k = 2 , s = 2)
      
    # conv4 layer 
    conv_w_4 = weight_variable([3 , 3 , 64 , 128]) 
    conv_b_4 = bias_variable([128])
    conv_output_4 = tf.nn.relu(conv2d(conv_pooling_3 , conv_w_4) + conv_b_4) 
    conv_bn_4 , conv_ema_4 = batch_norm_layer(conv_output_4 , on_train , True)
    
    # conv5 layer 
    conv_w_5 = weight_variable([3 , 3 , 128 , 128]) 
    conv_b_5 = bias_variable([128])
    conv_output_5 = tf.nn.relu(conv2d(conv_bn_4 , conv_w_5) + conv_b_5) 
    conv_bn_5 , conv_ema_5 = batch_norm_layer(conv_output_5 , on_train , True)
    conv_pooling_5 = max_pool(conv_bn_5 , k = 2 , s = 2)
    conv_pooling_5_flatten = tf.layers.flatten(conv_pooling_5)
    
    # fully connected layer 1
    fc_w_1 = weight_variable([conv_pooling_5_flatten.shape[1].value , 256])
    fc_b_1 = bias_variable([256])
    fc_output_1 = tf.nn.relu(tf.matmul(conv_pooling_5_flatten , fc_w_1) + fc_b_1)
    fc_bn_1 , fc_ema_1 = batch_norm_layer(fc_output_1 , on_train , False)
    fc_dropout_1 = tf.cond(on_train , 
                           lambda : tf.nn.dropout(fc_bn_1 , keep_prob = 0.5) , 
                           lambda : tf.nn.dropout(fc_bn_1 , keep_prob = 1))
       
    # fully connected layer 2
    fc_w_2 = weight_variable([256 , 3])
    fc_b_2 = bias_variable([3])
    prediction = tf.nn.softmax(tf.matmul(fc_dropout_1 , fc_w_2) + fc_b_2)
       
    ys_clothing = ys[: , :3]
    cross_entropy = ys_clothing * tf.log(prediction + 1e-9)  
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(cross_entropy , axis = 1))
    correct = tf.equal(tf.cast(tf.greater_equal(prediction , 0.5) , tf.int32) , tf.cast(ys_clothing , tf.int32))
    accuracy = tf.reduce_mean(tf.reduce_min(tf.cast(correct , tf.float32) , 1))
    
    ema_list = [conv_ema_1 , conv_ema_2 , conv_ema_3 , conv_ema_4 , conv_ema_5 , fc_ema_1]
    update_ema = tf.group(ema_list)
    optimizer = tf.train.AdamOptimizer(lr)
    grads_and_vars = optimizer.compute_gradients(cross_entropy)
    train_op = optimizer.apply_gradients(grads_and_vars)

    return cross_entropy , accuracy , update_ema , train_op

In [8]:
with tf.variable_scope('classfication_for_color'):
    cross_entropy_color , \
    accuracy_color , \
    update_ema_color , \
    train_op_color = classfication_for_color(xs , ys , on_train , lr)

with tf.variable_scope('classfication_for_clothing'):
    cross_entropy_clothing , \
    accuracy_clothing , \
    update_ema_clothing , \
    train_op_clothing = classfication_for_clothing(xs , ys , on_train , lr)    

In [9]:
# 數據增強
aug = ImageDataGenerator(rotation_range = 25, 
                         width_shift_range = 0.1,
                         height_shift_range = 0.1 ,
                         shear_range = 0.2 ,
                         zoom_range = 0.2,
                         horizontal_flip = True ,
                         fill_mode = 'nearest')

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

batches = 0
learning_rate = LR
train_loss_mean_record = {'color' : [] , 'clothing' : []}
val_loss_record = {'color' : [] , 'clothing' : []}
for epoch_i in range(0 , EPOCHS):
    train_loss_temp = {'color' : [] , 'clothing' : []}
    batches = 0
    for batch_i , (x_batch , y_batch) in enumerate(aug.flow(trainX , trainY , shuffle = True , batch_size = 32)):
        feed_dict = {xs : x_batch , 
                     ys : y_batch , 
                     on_train : True , 
                     lr : learning_rate}
        train_loss_color , train_acc_color , _ , _ = sess.run([cross_entropy_color , accuracy_color , train_op_color , update_ema_color] , feed_dict)
        train_loss_clothing , train_acc_clothing , _ , _ = sess.run([cross_entropy_clothing , accuracy_clothing , train_op_clothing , update_ema_clothing] , feed_dict)
        
        train_loss_temp['color'].append(train_loss_color)
        train_loss_temp['clothing'].append(train_loss_clothing)
        
        batches += 1
        if batches >= len(trainX) / 32: break
    
        if batch_i % 5 == 0: 
            print('=' * 30)
            print('epoch_i : {}'.format(epoch_i))
            print('batch_i : {}'.format(batch_i))
            print('train_loss_color : {:.2f} , train_loss_clothing : {:.2f}'.format(train_loss_color , train_loss_clothing))
            print('train_accuracy_color : {:.2%} , train_accuracy_clothing : {:.2%}\n'.format(train_acc_color , train_acc_clothing))
            
    feed_dict = {xs : testX , ys : testY , on_train : False}
    val_loss_clothing , val_acc_clothing = sess.run([cross_entropy_clothing , accuracy_clothing] , feed_dict)    
    val_loss_color , val_acc_color = sess.run([cross_entropy_color, accuracy_color] , feed_dict)
    
    print('*' * 30) 
    print('epoch_i : {}'.format(epoch_i))               
    print('train_loss_batch_mean_color : {:.2f}'.format(np.array(train_loss_temp['color']).mean()))
    print('train_loss_batch_mean_clothing : {:.2f}'.format(np.array(train_loss_temp['clothing']).mean()))
    print('val_accuracy_color : {:.2%}'.format(val_acc_color))    
    print('val_loss_clothing : {:.2f}'.format(val_loss_clothing))
    print('*' * 30 , '\n') 
    
    train_loss_mean_record['clothing'].append(np.array(train_loss_temp['clothing']).mean())
    train_loss_mean_record['color'].append(np.array(train_loss_temp['color']).mean())
    val_loss_record['clothing'].append(val_loss_clothing)
    val_loss_record['color'].append(val_loss_color)

epoch_i : 0
batch_i : 0
train_loss_color : 1.10 , train_loss_clothing : 1.10
train_accuracy_color : 0.00% , train_accuracy_clothing : 0.00%

epoch_i : 0
batch_i : 5
train_loss_color : 0.93 , train_loss_clothing : 0.99
train_accuracy_color : 0.00% , train_accuracy_clothing : 6.25%

epoch_i : 0
batch_i : 10
train_loss_color : 0.68 , train_loss_clothing : 0.92
train_accuracy_color : 56.25% , train_accuracy_clothing : 21.88%

epoch_i : 0
batch_i : 15
train_loss_color : 0.55 , train_loss_clothing : 0.56
train_accuracy_color : 75.00% , train_accuracy_clothing : 68.75%

epoch_i : 0
batch_i : 20
train_loss_color : 0.26 , train_loss_clothing : 0.54
train_accuracy_color : 93.75% , train_accuracy_clothing : 65.62%

epoch_i : 0
batch_i : 25
train_loss_color : 0.30 , train_loss_clothing : 0.46
train_accuracy_color : 81.25% , train_accuracy_clothing : 81.25%

epoch_i : 0
batch_i : 30
train_loss_color : 0.28 , train_loss_clothing : 0.29
train_accuracy_color : 87.50% , train_accuracy_clothing : 90.62%

******************************
epoch_i : 3
train_loss_batch_mean_color : 0.10
train_loss_batch_mean_clothing : 0.24
val_accuracy_color : 98.85%
val_loss_clothing : 0.16
****************************** 

epoch_i : 4
batch_i : 0
train_loss_color : 0.09 , train_loss_clothing : 0.45
train_accuracy_color : 100.00% , train_accuracy_clothing : 84.38%

epoch_i : 4
batch_i : 5
train_loss_color : 0.02 , train_loss_clothing : 0.34
train_accuracy_color : 100.00% , train_accuracy_clothing : 93.75%

epoch_i : 4
batch_i : 10
train_loss_color : 0.02 , train_loss_clothing : 0.27
train_accuracy_color : 100.00% , train_accuracy_clothing : 90.62%

epoch_i : 4
batch_i : 15
train_loss_color : 0.07 , train_loss_clothing : 0.30
train_accuracy_color : 96.88% , train_accuracy_clothing : 87.50%

epoch_i : 4
batch_i : 20
train_loss_color : 0.03 , train_loss_clothing : 0.31
train_accuracy_color : 100.00% , train_accuracy_clothing : 84.38%

epoch_i : 4
batch_i : 25
train_loss_color : 0.26 , train_loss_clothing : 0.1

epoch_i : 7
batch_i : 50
train_loss_color : 0.06 , train_loss_clothing : 0.04
train_accuracy_color : 96.88% , train_accuracy_clothing : 100.00%

******************************
epoch_i : 7
train_loss_batch_mean_color : 0.13
train_loss_batch_mean_clothing : 0.16
val_accuracy_color : 98.38%
val_loss_clothing : 0.12
****************************** 

epoch_i : 8
batch_i : 0
train_loss_color : 0.03 , train_loss_clothing : 0.18
train_accuracy_color : 100.00% , train_accuracy_clothing : 93.75%

epoch_i : 8
batch_i : 5
train_loss_color : 0.08 , train_loss_clothing : 0.23
train_accuracy_color : 96.88% , train_accuracy_clothing : 87.50%

epoch_i : 8
batch_i : 10
train_loss_color : 0.16 , train_loss_clothing : 0.54
train_accuracy_color : 93.75% , train_accuracy_clothing : 87.50%

epoch_i : 8
batch_i : 15
train_loss_color : 0.02 , train_loss_clothing : 0.18
train_accuracy_color : 100.00% , train_accuracy_clothing : 93.75%

epoch_i : 8
batch_i : 20
train_loss_color : 0.20 , train_loss_clothing : 0.15

epoch_i : 11
batch_i : 40
train_loss_color : 0.09 , train_loss_clothing : 0.32
train_accuracy_color : 96.88% , train_accuracy_clothing : 90.62%

epoch_i : 11
batch_i : 45
train_loss_color : 0.03 , train_loss_clothing : 0.20
train_accuracy_color : 100.00% , train_accuracy_clothing : 96.88%

epoch_i : 11
batch_i : 50
train_loss_color : 1.38 , train_loss_clothing : 0.10
train_accuracy_color : 78.12% , train_accuracy_clothing : 96.88%

******************************
epoch_i : 11
train_loss_batch_mean_color : 0.13
train_loss_batch_mean_clothing : 0.18
val_accuracy_color : 98.38%
val_loss_clothing : 0.14
****************************** 

epoch_i : 12
batch_i : 0
train_loss_color : 0.04 , train_loss_clothing : 0.07
train_accuracy_color : 100.00% , train_accuracy_clothing : 96.88%

epoch_i : 12
batch_i : 5
train_loss_color : 0.07 , train_loss_clothing : 0.32
train_accuracy_color : 96.88% , train_accuracy_clothing : 90.62%

epoch_i : 12
batch_i : 10
train_loss_color : 0.07 , train_loss_clothing 

epoch_i : 15
batch_i : 30
train_loss_color : 0.01 , train_loss_clothing : 0.24
train_accuracy_color : 100.00% , train_accuracy_clothing : 93.75%

epoch_i : 15
batch_i : 35
train_loss_color : 0.06 , train_loss_clothing : 0.05
train_accuracy_color : 96.88% , train_accuracy_clothing : 100.00%

epoch_i : 15
batch_i : 40
train_loss_color : 0.14 , train_loss_clothing : 0.29
train_accuracy_color : 96.88% , train_accuracy_clothing : 93.75%

epoch_i : 15
batch_i : 45
train_loss_color : 0.07 , train_loss_clothing : 0.08
train_accuracy_color : 96.88% , train_accuracy_clothing : 96.88%

epoch_i : 15
batch_i : 50
train_loss_color : 0.04 , train_loss_clothing : 0.17
train_accuracy_color : 100.00% , train_accuracy_clothing : 93.75%

******************************
epoch_i : 15
train_loss_batch_mean_color : 0.08
train_loss_batch_mean_clothing : 0.20
val_accuracy_color : 98.85%
val_loss_clothing : 0.17
****************************** 

epoch_i : 16
batch_i : 0
train_loss_color : 0.06 , train_loss_clothin

...

epoch_i : 30
batch_i : 50
train_loss_color : 0.03 , train_loss_clothing : 0.39
train_accuracy_color : 100.00% , train_accuracy_clothing : 84.38%

******************************
epoch_i : 30
train_loss_batch_mean_color : 0.10
train_loss_batch_mean_clothing : 0.19
val_accuracy_color : 98.61%
val_loss_clothing : 0.15
****************************** 

epoch_i : 31
batch_i : 0
train_loss_color : 0.02 , train_loss_clothing : 0.04
train_accuracy_color : 100.00% , train_accuracy_clothing : 100.00%

epoch_i : 31
batch_i : 5
train_loss_color : 0.02 , train_loss_clothing : 0.12
train_accuracy_color : 100.00% , train_accuracy_clothing : 93.75%

epoch_i : 31
batch_i : 10
train_loss_color : 0.12 , train_loss_clothing : 0.05
train_accuracy_color : 96.88% , train_accuracy_clothing : 100.00%

epoch_i : 31
batch_i : 15
train_loss_color : 0.05 , train_loss_clothing : 0.09
train_accuracy_color : 96.88% , train_accuracy_clothing : 96.88%

epoch_i : 31
batch_i : 20
train_loss_color : 0.06 , train_loss_clothi

epoch_i : 34
batch_i : 40
train_loss_color : 0.09 , train_loss_clothing : 0.08
train_accuracy_color : 93.75% , train_accuracy_clothing : 96.88%

epoch_i : 34
batch_i : 45
train_loss_color : 0.05 , train_loss_clothing : 0.03
train_accuracy_color : 100.00% , train_accuracy_clothing : 100.00%

epoch_i : 34
batch_i : 50
train_loss_color : 0.03 , train_loss_clothing : 0.17
train_accuracy_color : 100.00% , train_accuracy_clothing : 93.75%

******************************
epoch_i : 34
train_loss_batch_mean_color : 0.12
train_loss_batch_mean_clothing : 0.12
val_accuracy_color : 95.61%
val_loss_clothing : 0.10
****************************** 

epoch_i : 35
batch_i : 0
train_loss_color : 0.04 , train_loss_clothing : 0.11
train_accuracy_color : 100.00% , train_accuracy_clothing : 93.75%

epoch_i : 35
batch_i : 5
train_loss_color : 0.23 , train_loss_clothing : 0.21
train_accuracy_color : 87.50% , train_accuracy_clothing : 93.75%

epoch_i : 35
batch_i : 10
train_loss_color : 0.13 , train_loss_clothin

epoch_i : 38
batch_i : 30
train_loss_color : 0.21 , train_loss_clothing : 0.03
train_accuracy_color : 93.75% , train_accuracy_clothing : 100.00%

epoch_i : 38
batch_i : 35
train_loss_color : 0.17 , train_loss_clothing : 0.07
train_accuracy_color : 90.62% , train_accuracy_clothing : 96.88%

epoch_i : 38
batch_i : 40
train_loss_color : 0.01 , train_loss_clothing : 0.03
train_accuracy_color : 100.00% , train_accuracy_clothing : 100.00%

epoch_i : 38
batch_i : 45
train_loss_color : 0.03 , train_loss_clothing : 0.03
train_accuracy_color : 100.00% , train_accuracy_clothing : 100.00%

epoch_i : 38
batch_i : 50
train_loss_color : 0.06 , train_loss_clothing : 0.05
train_accuracy_color : 96.88% , train_accuracy_clothing : 96.88%

******************************
epoch_i : 38
train_loss_batch_mean_color : 0.10
train_loss_batch_mean_clothing : 0.12
val_accuracy_color : 98.61%
val_loss_clothing : 0.08
****************************** 

epoch_i : 39
batch_i : 0
train_loss_color : 0.22 , train_loss_cloth

epoch_i : 42
batch_i : 20
train_loss_color : 0.04 , train_loss_clothing : 0.08
train_accuracy_color : 100.00% , train_accuracy_clothing : 96.88%

epoch_i : 42
batch_i : 25
train_loss_color : 0.22 , train_loss_clothing : 0.08
train_accuracy_color : 93.75% , train_accuracy_clothing : 93.75%

epoch_i : 42
batch_i : 30
train_loss_color : 0.12 , train_loss_clothing : 0.05
train_accuracy_color : 93.75% , train_accuracy_clothing : 96.88%

epoch_i : 42
batch_i : 35
train_loss_color : 0.13 , train_loss_clothing : 0.01
train_accuracy_color : 96.88% , train_accuracy_clothing : 100.00%

epoch_i : 42
batch_i : 40
train_loss_color : 0.13 , train_loss_clothing : 0.04
train_accuracy_color : 96.88% , train_accuracy_clothing : 96.88%

epoch_i : 42
batch_i : 45
train_loss_color : 0.20 , train_loss_clothing : 0.09
train_accuracy_color : 93.75% , train_accuracy_clothing : 96.88%

epoch_i : 42
batch_i : 50
train_loss_color : 0.01 , train_loss_clothing : 0.15
train_accuracy_color : 100.00% , train_accuracy_c

epoch_i : 46
batch_i : 10
train_loss_color : 0.06 , train_loss_clothing : 0.16
train_accuracy_color : 100.00% , train_accuracy_clothing : 96.88%

epoch_i : 46
batch_i : 15
train_loss_color : 0.06 , train_loss_clothing : 0.11
train_accuracy_color : 96.88% , train_accuracy_clothing : 93.75%

epoch_i : 46
batch_i : 20
train_loss_color : 0.04 , train_loss_clothing : 0.07
train_accuracy_color : 100.00% , train_accuracy_clothing : 96.88%

epoch_i : 46
batch_i : 25
train_loss_color : 0.02 , train_loss_clothing : 0.25
train_accuracy_color : 100.00% , train_accuracy_clothing : 90.62%

epoch_i : 46
batch_i : 30
train_loss_color : 0.24 , train_loss_clothing : 0.22
train_accuracy_color : 93.75% , train_accuracy_clothing : 96.88%

epoch_i : 46
batch_i : 35
train_loss_color : 0.07 , train_loss_clothing : 0.03
train_accuracy_color : 100.00% , train_accuracy_clothing : 100.00%

epoch_i : 46
batch_i : 40
train_loss_color : 0.13 , train_loss_clothing : 0.08
train_accuracy_color : 93.75% , train_accuracy